In [1]:
import pandas as pd
import rasterio
import geopandas as gpd
import geemap
import ee
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import fiona
from fiona import listlayers
import geojson
import json
import os

**Landsat-8**

In [3]:
# Function to calculate NDVI
def calculate_ndvi(image):
      nir = image.select('B5')
      red = image.select('B4')
      ndvi = image.expression(
          '(NIR-RED)/(NIR+RED)',
          {
              'NIR': nir,
              'RED': red
          }
      ).rename('NDVI')
      return ndvi
# Function to calculate Wetness
def calculate_wet(image):
  blue = image.select('B2')
  green = image.select('B3')
  red = image.select('B4')
  nir = image.select('B5')
  swir1 = image.select('B6')
  swir2 = image.select('B7')
  wetness = image.expression(
      '(0.1511*BLUE) +(0.1973*GREEN)+(0.3283*RED)+(0.3407*NIR)-(0.7117*SWIR1)-(0.4559*SWIR2)'
      , {
          'BLUE': blue,
          'GREEN': green,
          'RED': red,
          'NIR': nir,
          'SWIR1': swir1,
          'SWIR2': swir2
      }).rename('Wetness')
  return wetness
# Function to calculate NDBSI
def calculate_ndbsi(image):
  blue = image.select('B2')
  green = image.select('B3')
  red = image.select('B4')
  nir = image.select('B5')
  swir1= image.select('B6')
  swir2 = image.select('B7')
  si= image.expression(
      '((SWIR1+RED)-(NIR+BLUE))/((SWIR1+RED)+(NIR+BLUE))',
       {
          'NIR': nir,
          'RED': red,
          'GREEN': green,
          'SWIR1': swir1,
          'SWIR2': swir2,
          'BLUE': blue
      }).rename('SI')
  ibi = image.expression(
      '(2 * SWIR1) / (SWIR1 + NIR) - [(NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))]/(2 * SWIR1) / (SWIR1 + NIR) + [(NIR / (NIR + RED)) + (GREEN / (GREEN + SWIR1))]', {
          'NIR': nir,
          'RED': red,
          'GREEN': green,
          'SWIR1': swir1,
          'SWIR2': swir2
      }).rename('IBI')
  ndbsi = image.expression(
      '(IBI+SI)/2', {
          'IBI': ibi,
          'SI': si
      }).rename('NDBSI')
  return ndbsi


# Function to calculate LST
def calculate_lst(image):
  # Step 1: Calculate NDVI
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')

  # Step 2: Calculate minimum and maximum NDVI in the AOI
  ndviMin = ee.Number(ndvi.reduceRegion(
          reducer=ee.Reducer.min(),
          geometry=aoi,
          scale=30,
          maxPixels=1e9
      ).values().get(0))

  ndviMax = ee.Number(ndvi.reduceRegion(
          reducer=ee.Reducer.max(),
          geometry=aoi,
          scale=30,
          maxPixels=1e9
      ).values().get(0))

  # Step 3: Calculate Fractional Vegetation (FV)
  fv = ndvi.subtract(ndviMin) \
              .divide(ndviMax.subtract(ndviMin)) \
              .pow(ee.Number(2)) \
              .rename('FV')

  # Step 4: Calculate Emissivity (EM)
  em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

  # Step 5: Select Thermal Band (Band 10)
  thermal = image.select('B10').rename('thermal')

  # Step 6: Calculate Land Surface Temperature (LST)
  lst = thermal.expression(
          '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
              'TB': thermal.select('thermal'),  # Brightness temperature in Kelvin
              'em': em  # Emissivity
          }).rename('LST')
  return lst

**Image Export**

In [5]:
def export_image(image, image_name):
      # Create and start the export task
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f"{image_name}",
            folder="GEE_Exports",
            fileNamePrefix=image_name,
            region=aoi,
            fileFormat='GeoTIFF',
            scale=30,
            maxPixels=1e13
        )
        task.start()
        print(f"Export task for '{image_name}' started. Check Google Drive for output.")

In [ ]:
projectid = input("Enter your google engine cloud project id: ")

In [24]:
ee.Authenticate()
ee.Initialize(project= projectid)
Map = geemap.Map(center=(35.12107930404553, -89.98483533026703), zoom=9)
Map

Map(center=[23.715081709283623, 90.08962659013804], controls=(WidgetControl(options=['position', 'transparent_…

In [30]:
#Location of your shapefile
filePath = r"C:\Users\USER\OneDrive - The University of Memphis\U of M\Spring 2025\GIS Living Lab\UHI_Code\County_Shape\Shelby.shp"

In [32]:
gdf = gpd.read_file(filePath)
geojson_str = gdf.to_json()
geojson_dict = json.loads(geojson_str)
aoi = ee.FeatureCollection(geojson_dict).geometry()
Map.addLayer(aoi, {}, 'Area of Interest')

In [42]:
sImage = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
             .filterBounds(aoi) \
             .filterDate('2022-05-01', '2022-08-30') \
             .filterMetadata('CLOUD_COVER', 'less_than', 0.1) \

sImage8 = sImage.median().clip(aoi)
Map.addLayer(sImage8, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Landsat-8 Image')

In [44]:
# Convert metadata to a list
image_metadata = sImage.map(lambda img: img.select([]).set('id', img.id(), 'date', img.date().format(), 'cloud_cover', img.get('CLOUD_COVER')))
metadata_list = []

name = image_metadata.aggregate_array('id').getInfo()
metadata_list.append(name)
date = image_metadata.aggregate_array('date').getInfo()
metadata_list.append(date)
cloud_cover = image_metadata.aggregate_array('cloud_cover').getInfo()
metadata_list.append(cloud_cover)
# Print metadata
print("Metadata for images used in the composite:")
print(name)
print(date)
print(cloud_cover)

Metadata for images used in the composite:
['LC08_023035_20220711', 'LC08_023036_20220625']
['2022-07-11T16:36:55', '2022-06-25T16:37:19']
[0, 0.08]


In [46]:
ndvi = calculate_ndvi(sImage8)
wetness = calculate_wet(sImage8)
ndbsi = calculate_ndbsi(sImage8)
lst = calculate_lst(sImage8)

In [48]:
export_image(lst, "lst_22_Shelby")

Export task for 'lst_22_Shelby' started. Check Google Drive for output.


In [12]:
export_image(ndvi, "ndvi_22_Shelby")
export_image(wetness, "wetness_22_Shelby")
export_image(ndbsi, "ndbsi_22_Shelby")
export_image(lst, "lst_22_Shelby")

Export task for 'ndvi_22_Shelby' started. Check Google Drive for output.
Export task for 'wetness_22_Shelby' started. Check Google Drive for output.
Export task for 'ndbsi_22_Shelby' started. Check Google Drive for output.
Export task for 'lst_22_Shelby' started. Check Google Drive for output.
